<a href="https://colab.research.google.com/github/SamyKr/PIR_2024/blob/main/PIR_V2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CHARGEMENT DES MODULES

In [ ]:
# IMPORTATIONS DES MODULES
! pip install torch
!pip install gdown
! pip install options
! pip install segmentation-models-pytorch

TELECHARGEMENT DES DATAS + ORGANISATION DES DATAS

In [ ]:
import gdown
import os
import shutil
import zipfile
import os


# Liste des IDs des fichiers
file_ids = [
    '1BqSt4ueO7XAyQ_84mUjswUSJt13ZBuzG',  # val.zip
    '18GuoCuBn48oZKAlEo-LrNwABrFhVALU-',  # train.zip
    '1jj3qJD_grJlgIhUWO09zibRGJe0R4Tn0',  # test.zip
    '1QOjoa-YFvHOgCLXaxvZUlrlidihpmbWU',  # pam_net_F.pth
    '196bR8LdSfVawnSOVmM_YPW25UetucUYX',  # pam_net_A.pth
    '1IVdF5a3e1_7DiSndtMkhpZuCSgDLLFcg'   # best_ckpt.pt
]

# Liste des noms des fichiers
file_names = [
    'val.zip',
    'train.zip',
    'test.zip',
    'pam_net_F.pth',
    'pam_net_A.pth',
    'best_ckpt.pt'
]

# Télécharger chaque fichier en utilisant son ID
for file_id, file_name in zip(file_ids, file_names):
    gdown.download(f'https://drive.google.com/uc?id={file_id}', file_name, quiet=False)

# Créer les dossiers nécessaires
os.makedirs('LEVIR-CD/PAM-PRETRAIN', exist_ok=True)
os.makedirs('LEVIR-CD/BIT_LEVIR', exist_ok=True)
os.makedirs('LEVIR-CD/train', exist_ok=True)
os.makedirs('LEVIR-CD/test', exist_ok=True)
os.makedirs('LEVIR-CD/val', exist_ok=True)

# Déplacer les fichiers dans les sous-dossiers appropriés
shutil.move('pam_net_A.pth', 'LEVIR-CD/PAM-PRETRAIN/pam_net_A.pth')
shutil.move('pam_net_F.pth', 'LEVIR-CD/PAM-PRETRAIN/pam_net_F.pth')
shutil.move('best_ckpt.pt', 'LEVIR-CD/BIT_LEVIR/best_ckpt.pt')
shutil.move('val.zip', 'LEVIR-CD/val.zip')
shutil.move('train.zip', 'LEVIR-CD/train.zip')
shutil.move('test.zip', 'LEVIR-CD/test.zip')






def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f'Décompressé {zip_path} dans {extract_to}')

# Décompression des fichiers
unzip_file('LEVIR-CD/val.zip', 'LEVIR-CD/val')
unzip_file('LEVIR-CD/train.zip', 'LEVIR-CD/train')
unzip_file('LEVIR-CD/test.zip', 'LEVIR-CD/test')
os.remove('LEVIR-CD/val.zip')
os.remove('LEVIR-CD/train.zip')
os.remove('LEVIR-CD/test.zip')
print("Tous les fichiers ont été décompressés et supprimés")





print("Tous les fichiers ont été téléchargés et organisés.")


Downloading...
From (original): https://drive.google.com/uc?id=1BqSt4ueO7XAyQ_84mUjswUSJt13ZBuzG
From (redirected): https://drive.google.com/uc?id=1BqSt4ueO7XAyQ_84mUjswUSJt13ZBuzG&confirm=t&uuid=c8895bc9-e365-4347-b3dd-75e9e2b685cb
To: /content/val.zip
100%|██████████| 246M/246M [00:04<00:00, 51.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18GuoCuBn48oZKAlEo-LrNwABrFhVALU-
From (redirected): https://drive.google.com/uc?id=18GuoCuBn48oZKAlEo-LrNwABrFhVALU-&confirm=t&uuid=14113387-ed60-40b8-919a-3ab03f9067c3
To: /content/train.zip
100%|██████████| 1.72G/1.72G [00:25<00:00, 66.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1jj3qJD_grJlgIhUWO09zibRGJe0R4Tn0
From (redirected): https://drive.google.com/uc?id=1jj3qJD_grJlgIhUWO09zibRGJe0R4Tn0&confirm=t&uuid=e29dbb69-0aaa-48b2-b9e6-e49fa6dc1bae
To: /content/test.zip
100%|██████████| 496M/496M [00:11<00:00, 42.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QOjoa-YFvHOgCLXaxvZUlrlidih

Décompressé LEVIR-CD/val.zip dans LEVIR-CD/val
Décompressé LEVIR-CD/train.zip dans LEVIR-CD/train
Décompressé LEVIR-CD/test.zip dans LEVIR-CD/test
Tous les fichiers ont été décompressés et supprimés
Tous les fichiers ont été téléchargés et organisés.


In [11]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

class LEVIRCDClassificationDataset(Dataset):
    def __init__(self, img_dir_before, img_dir_after, mask_dir, transform=None):
        self.img_dir_before = img_dir_before
        self.img_dir_after = img_dir_after
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir_before)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_before_path = os.path.join(self.img_dir_before, img_name)
        img_after_path = os.path.join(self.img_dir_after, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        img_before = Image.open(img_before_path).convert("RGB")
        img_after = Image.open(img_after_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Grayscale mask

        # Calculer le label binaire : 1 s'il y a un changement, 0 sinon
        label = 1 if torch.sum(transforms.ToTensor()(mask)) > 0 else 0

        if self.transform:
            img_before = self.transform(img_before)
            img_after = self.transform(img_after)

        # Retourner les images avant, après, et le label binaire
        return torch.cat([img_before, img_after], dim=0), label, img_name

# Transforms pour normaliser les images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Chemins vers les répertoires de données
img_dir_before = '/content/LEVIR-CD/train/A'
img_dir_after = '/content/LEVIR-CD/train/B'
mask_dir = '/content/LEVIR-CD/train/label'

# Charger les données
train_dataset = LEVIRCDClassificationDataset(img_dir_before, img_dir_after, mask_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from sklearn.metrics import accuracy_score, precision_score, recall_score
import os
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt


# Vérifier si un GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Charger le modèle U-Net avec un backbone pré-entraîné (ResNet34)
def get_pretrained_unet():
    model = smp.Unet(
        encoder_name="resnet34",        # Utilise ResNet34 pré-entraîné sur ImageNet
        encoder_weights="imagenet",     # Poids pré-entraînés sur ImageNet
        in_channels=6,                  # Car deux images (avant/après) empilées = 6 canaux
        classes=1,                      # Une sortie binaire (détection de changement)
    )
    return model

# Dataset LEVIR-CD
class LEVIRCDDataset(torch.utils.data.Dataset):
    def __init__(self, imageA_dir, imageB_dir, label_dir, transform=None):
        self.imageA_dir = imageA_dir
        self.imageB_dir = imageB_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(imageA_dir))  # Suppose que les fichiers sont ordonnés

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_name = self.image_paths[idx]
        imageA = Image.open(os.path.join(self.imageA_dir, self.image_paths[idx])).convert('RGB')
        imageB = Image.open(os.path.join(self.imageB_dir, self.image_paths[idx])).convert('RGB')
        label = Image.open(os.path.join(self.label_dir, self.image_paths[idx])).convert('L')

        if self.transform:
            imageA = self.transform(imageA)
            imageB = self.transform(imageB)
            label = self.transform(label)

        # Concaténer les deux images pour obtenir 6 canaux (3 canaux de chaque image)
        combined_image = torch.cat([imageA, imageB], dim=0)
        return combined_image, label,image_name

# Configuration des hyperparamètres
num_epochs = 7
learning_rate = 0.001
batch_size = 8

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((1024, 1024)),
])

# Charger les données d'entraînement et de validation
train_dataset = LEVIRCDDataset('/content/LEVIR-CD/train/A', '/content/LEVIR-CD/train/B', '/content/LEVIR-CD/train/label', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = LEVIRCDDataset('/content/LEVIR-CD/val/A', '/content/LEVIR-CD/val/B', '/content/LEVIR-CD/val/label', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Créer le modèle U-Net
model = get_pretrained_unet().to(device)

# Fonction de perte et optimiseur
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Fonction d'entraînement
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Mettre le modèle en mode entraînement
        running_loss = 0.0
        for images, labels, _ in train_loader:
            images, labels = images.to(device), labels.to(device)  # Envoyer les données sur le GPU/CPU
            labels = labels.squeeze(1)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1).float())

            # Backward pass et optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")



# Entraîner le modèle
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)






Using device: cuda
Epoch [1/7], Loss: 0.3014
Epoch [2/7], Loss: 0.1023
Epoch [3/7], Loss: 0.0685
Epoch [4/7], Loss: 0.0586
Epoch [5/7], Loss: 0.0533
Epoch [6/7], Loss: 0.0454
Epoch [7/7], Loss: 0.0416


In [13]:


# Fonction de validation
def validate_model_and_save_predictions(model, val_loader, save_dir='predictions'):
    model.eval()
    all_preds = []
    all_labels = []

    # Créer un dossier pour sauvegarder les cartes si nécessaire
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    with torch.no_grad():  # Pas de calcul des gradients pendant l'évaluation
        for i, (images, labels,image_names) in enumerate(val_loader):
            images = images.to(device)  # Envoyer les images sur le GPU/CPU

            # Faire les prédictions
            outputs = model(images)
            preds = torch.sigmoid(outputs)  # Appliquer la fonction sigmoïde pour obtenir des probabilités
            preds = preds.cpu().numpy()

            # SAVE SHIFT DETECTION MAP
            for j in range(preds.shape[0]):
                img_name=image_names[j]
                pred = preds[j, 0]  # Extraire la carte de changement pour la jème image du batch
                plt.imsave(os.path.join(save_dir, f'change_map_{img_name}'), pred, cmap='gray')




    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    print(f"Validation Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"Cartes de détection de changement enregistrées dans le dossier : {save_dir}")


validate_model_and_save_predictions(model, val_loader)

Validation Accuracy: nan, Precision: 0.0000, Recall: 0.0000
Cartes de détection de changement enregistrées dans le dossier : predictions


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Appliquer la dilatation et l'érosion
def apply_morphological_operations(pred, kernel_size=5):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Appliquer la dilatation puis l'érosion
    dilated = cv2.dilate(pred, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)

    return eroded

# Convertir tous les pixels non noirs en blanc
def convert_non_black_to_white(pred):
    pred[pred > 40] = 255 #40 c'est pour la tolérance
    return pred

# Fonction pour traiter les prédictions existantes
def process_saved_predictions(input_dir='predictions', output_dir='processed_predictions', kernel_size=5):
    # Créer un dossier pour sauvegarder les résultats traités
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Lister tous les fichiers de prédictions sauvegardés
    pred_files = sorted(os.listdir(input_dir))

    for pred_file in pred_files:
        # Charger l'image prédiction
        pred_path = os.path.join(input_dir, pred_file)
        pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)  # Lire en mode niveaux de gris

        # Appliquer les opérations morphologiques
        pred = apply_morphological_operations(pred, kernel_size=kernel_size)

        # Convertir les pixels non noirs en blanc
        pred = convert_non_black_to_white(pred)

        # Sauvegarder l'image traitée
        output_path = os.path.join(output_dir,'p_'+ pred_file)
        plt.imsave(output_path, pred, cmap='gray')

    print(f"Prédictions traitées et sauvegardées dans le dossier : {output_dir}")

# Appliquer le traitement sur les images de prédiction existantes
process_saved_predictions(input_dir='predictions', output_dir='processed_predictions')


Prédictions traitées et sauvegardées dans le dossier : processed_predictions


In [30]:
from google.colab import files


files.download('LEVIR-CD/val/label/val_11.png')
files.download('LEVIR-CD/val/label/val_12.png')
files.download('LEVIR-CD/val/label/val_13.png')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>